In [18]:
import numpy as np
import pandas as pd
from scipy.spatial import cKDTree
import random
import pandas as pd
import numpy as np
import os
import requests
import functools
import matplotlib.pyplot as plt
import plotly.express as px
from multiprocessing.pool import ThreadPool
from tenacity import retry, stop_after_attempt, wait_exponential

# Create a session object for HTTP requests
session = requests.Session()
# Create ThreadPool for concurrent requests
pool = ThreadPool()


# Read your latitude and longitude data from a CSV file (replace 'Long_Lats.csv' with your file)
df = pd.read_csv("./data/bangkok/optimize/patients.csv", delimiter=',', skiprows=0, low_memory=False)
rendezvous_locations = pd.read_csv('./all_outputs/rendezvous_locations_within_30_min_from_msu.csv').to_numpy()
msus = pd.read_csv('./data/bangkok/optimize/msus.csv', delimiter=',', skiprows=0, low_memory=False)
df.drop(columns=['count'], inplace=True)
patient_locations = df.to_numpy()[:, 1:]
msu_location = msus.to_numpy()[:, 1:]
msu_kdtree = cKDTree(msu_location)
patients_kdtree = cKDTree(patient_locations)


In [19]:

# Define a fallback function to return a high travel time in case of retries exhaustion
def retry_fallback(retry_state):
    # You can log the failure or perform other actions here
    print("Retries exhausted. Returning a fallback travel time.")
    return 1000  # Fallback travel time in minutes

@functools.lru_cache(maxsize=4096)  # cache up to 4096 unique routes (can be adjusted as needed)
@retry(stop=stop_after_attempt(10), wait=wait_exponential(multiplier=1, min=2, max=30),
       retry_error_callback=retry_fallback)
def cached_travel_time(c1, c2):
    # OSRM API endpoint for routing
    osrm_endpoint = "http://139.59.32.22:5000/route/v1/driving/"
    # Format coordinates for the API request
    coordinates = f"{c1[1]},{c1[0]};{c2[1]},{c2[0]}"
    # Construct the full URL for the API request
    url = osrm_endpoint + coordinates + "?overview=false"

    # Use the global session object for making the request
    response = session.get(url, timeout=10)  # Adding a timeout for good measure
    data = response.json()

    # Check if the response contains valid routing data
    if data['code'] == 'Ok':
        # Extract travel time from the first (and only) route
        travel_time_seconds = data['routes'][0]['duration']
        # Convert travel time to minutes
        travel_time_minutes = travel_time_seconds / 60
        return travel_time_minutes
    else:
        # Return a high value or handle the error appropriately if no route is found
        return 1000  # Example fallback value

# Calculate travel time based on distance and speed assumption
def calculate_travel_time(c1, c2):
    return cached_travel_time(tuple(c1), tuple(c2))

# min30_indices=[]
# for index in range(len(rendezvous_locations)):
#     msu_to_rendevu = calculate_travel_time(msu_location[0], rendezvous_locations[index])
#     if msu_to_rendevu <= 30 and msu_to_rendevu != 0:
#         min30_indices.append(index)
# pd.DataFrame(rendezvous_locations[min30_indices], columns=['longitude', 
#                                                            'latitude']).to_csv(
#                                                                f'{OUTPUT_PATH}rendezvous_locations_within_30_min_from_msu.csv', 
#                                                                index=False)

In [95]:
POPULATION_SIZE = 100
NUM_SITES=5
NUM_GENERATIONS = 100
TOURNAMENT_SIZE=4
MUTATION_RATE = 0.12
CROSSOVER_RATE = 0.4
OUTPUT_PATH='all_outputs1/'
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)
pd.DataFrame([[NUM_SITES, CROSSOVER_RATE,
              NUM_GENERATIONS, POPULATION_SIZE, 
              TOURNAMENT_SIZE]], columns=['NUM_SITES', 
                                         'CROSSOVER_RATE',
                                         'NUM_GENERATIONS',
                                         'POPULATION_SIZE',
                                         'TOURNAMENT_SIZE']).to_csv(f'{OUTPUT_PATH}best_parameters_st.csv')

In [92]:
# Initialize population
def initialize_population():
    return [random.sample(range(len(rendezvous_locations))
                          , NUM_SITES) for _ in range(POPULATION_SIZE)]

#optimize the fitness calculation to sum of only nearby 10 patients 
def calculate_population_fitness(population):
    population_weights = []
    for chromosome in population:
        chromosome_kdtree = cKDTree(rendezvous_locations[chromosome])
        nearest_rendevu_for_each_patient_dist, nearest_rendevu_for_each_patient_idx = chromosome_kdtree.query(patient_locations, k=1)
        nearest_msu_for_each_patient_dist, _ = msu_kdtree.query(patient_locations, k=1)
        
        compare = nearest_rendevu_for_each_patient_dist < nearest_msu_for_each_patient_dist
        this_rendevu_to_nearest_msu, _ = msu_kdtree.query(rendezvous_locations[nearest_rendevu_for_each_patient_idx], k=1)
        chromosome_weight = np.where(compare, nearest_rendevu_for_each_patient_dist + this_rendevu_to_nearest_msu, 
                                     nearest_msu_for_each_patient_dist).sum()
        # chromosome_weight = (nearest_rendevu_for_each_patient_dist + this_rendevu_to_nearest_msu).sum() 
        population_weights.append(chromosome_weight)
    
    return population_weights

#crossover 2 parents when sampling is less than crossover_rate
def crossOver(parent1, parent2, crossover_rate):
    if random.random() < crossover_rate:
        # crossover_point = random.randint(1, len(parent1)-1)
        crossover_point = 3
        return parent1[:crossover_point] + parent2[crossover_point:], parent2[:crossover_point] + parent1[crossover_point:]
    else:
        return parent1, parent2
    
#from nearby_rendevu_indices, get a list of indices 
# that are within 30minutes from msu dispatch loc
min30_indices=range(len(rendezvous_locations))

#muate each gene of chromosome when if sampling for that gene is < mutation_rate
def mutate(chromosome, mutation_rate):
    new_chromosome = []
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            new_chromosome.append(random.sample(min30_indices, k=1)[0])
        else:
            new_chromosome.append(chromosome[i])
    return new_chromosome

def tournament_selection(population):
    one_gen = []
    for _ in range(len(population)//2):
        rando4sample=random.sample(population, TOURNAMENT_SIZE)
        rando4sampel_fitness=calculate_population_fitness(rando4sample)
        best2 = np.argsort(rando4sampel_fitness)[:2]
        crossed_best2 = crossOver(rando4sample[best2[0]],rando4sample[best2[1]],CROSSOVER_RATE)
        child1 = mutate(crossed_best2[0], MUTATION_RATE)
        child2 = mutate(crossed_best2[1], MUTATION_RATE)
        one_gen.append(child1)
        one_gen.append(child2)
    return one_gen

def main():
    initial_popu = initialize_population()
    each_gen_fitness_score = []
    each_gen_best_chromosome = []
    each_gen_best_chromosome_fitness_score = []
    for i in range(NUM_GENERATIONS):
        print(f'Generation no: {i}')
        current_fitness_of_population = calculate_population_fitness(initial_popu)
        current_fitness=np.sum(current_fitness_of_population)
        each_gen_fitness_score.append(current_fitness)
        print(f'current_avg_fitness_of_population: {current_fitness}')
        best_chromosome_idx=np.argsort(current_fitness_of_population)[0]
        best_chromosome = initial_popu[best_chromosome_idx]
        each_gen_best_chromosome.append(best_chromosome)
        best_chromosome_fitness = current_fitness_of_population[best_chromosome_idx]
        each_gen_best_chromosome_fitness_score.append(np.sum(best_chromosome_fitness))
        print(f'current_best_potential_sites_fitness: {np.sum(best_chromosome_fitness)}')
        next_gen = tournament_selection(initial_popu)
        next_gen_fitness = np.sum(calculate_population_fitness(next_gen))
        if next_gen_fitness < current_fitness:
            initial_popu = next_gen
    final_gen = initial_popu
    return final_gen, each_gen_fitness_score, each_gen_best_chromosome, each_gen_best_chromosome_fitness_score
        
def handle_results(last_gen):
    best_rps_df=pd.DataFrame([rendezvous_locations[last_gen][i] for i in range(5)], 
                         columns=['longitude', 'latitude'])
    #save best rendezvous locations to csv
    best_rps_df.to_csv(f'{OUTPUT_PATH}best{NUM_SITES}_rendezvous_sites.csv',
                       index=False)
    df = pd.read_csv(f'{OUTPUT_PATH}best{NUM_SITES}_rendezvous_sites.csv')
    df=df.to_numpy()
    rps_kdtree = cKDTree(df)
    #from the perspective of patient, find the nearest rendevu site
    _, nearest_rps_for_each_patient_idx = rps_kdtree.query(patient_locations,k=1)
    #from the perspective of rendevu, find the nearest msu
    _, nearest_msu_for_each_rendevu = msu_kdtree.query(df, k=1)
    travel_time_bw_rendevu_and_msu=[round(calculate_travel_time(df[i], 
                      msu_location[nearest_msu_for_each_rendevu][i]),
                                      4) for i in range(len(df))]
    patient_locations_df = pd.DataFrame(patient_locations, columns=['patient_longitude',
                                                                    'patient_latitude'])
    nearest_rps_for_each_patient_df=pd.DataFrame(df[nearest_rps_for_each_patient_idx], 
                                                columns=['rendevu_longitude',
                                                        'rendevu_latitude'])
    nearest_msu_for_each_rendevu_df=pd.DataFrame(msu_location[nearest_msu_for_each_rendevu], 
                columns=['nearby_msu_longitude',
                        'nearby_msu_latitude'])
    travel_time_bw_rendevu_and_msu_df=pd.DataFrame(travel_time_bw_rendevu_and_msu, 
             columns=['time_in_mins'])
    nearest_msu_for_each_rendevu_df=pd.concat([best_rps_df, nearest_msu_for_each_rendevu_df,
           travel_time_bw_rendevu_and_msu_df], axis=1)
    travel_time_bw_patient_nearby_rendevu=[round(calculate_travel_time(patient_locations[i], 
                      df[nearest_rps_for_each_patient_idx][i]),
                                      4) for i in range(
                                          len(patient_locations))]
    travel_time_bw_patient_nearby_rendevu_df=pd.DataFrame(travel_time_bw_patient_nearby_rendevu,
                                                      columns=['time_in_mins'])
    nearest_msu_for_each_rendevu_df.to_csv(f'{OUTPUT_PATH}nearest_msu_for_each_rendevu.csv')
    nearest_rps_for_each_patient=pd.concat([patient_locations_df, nearest_rps_for_each_patient_df, 
                                            travel_time_bw_patient_nearby_rendevu_df], axis=1)
    nearest_rps_for_each_patient.to_csv(f'{OUTPUT_PATH}nearest_rps_for_each_patient.csv')
        

In [ ]:
#first complete run
a,b,c,d = main()

In [ ]:
#save the results to csv
handle_results(last_gen=c[-1])

In [ ]:
#plotting each generation's fitness score
plt.plot(b)
plt.title('Each_gen_fitness_score')
plt.xlabel('generation_no:')
plt.ylabel('fitness score')

In [ ]:
fig = px.scatter_geo(patient_locations, 
                     lat=patient_locations[:, 0], 
                     lon=patient_locations[:, 1], 
                     color_discrete_sequence=['green'])
fig.add_trace(px.scatter_geo(rendezvous_locations,
                             lat=rendezvous_locations[:, 0],
                             lon=rendezvous_locations[:, 1],
                             color_discrete_sequence=['blue']).data[0])
fig.add_trace(px.scatter_geo(msu_location,
                             lat=msu_location[:, 0],
                             lon=msu_location[:, 1],
                             color_discrete_sequence=['yellow']).data[0])
fig.add_trace(px.scatter_geo(msu_location,
                             lat=rendezvous_locations[c[-1]][:, 0],
                             lon=rendezvous_locations[c[-1]][:, 1],
                             color_discrete_sequence=['red']).data[0])

fig.update_layout(
    autosize=False,
    width=1100,
    height=800,
    paper_bgcolor="LightSteelBlue"
)

fig.show()